In [1]:
import re
import random
import numpy as np
import pandas as pd
from collections import Counter  

In [2]:
def random_pick_1(seq,probabilities):
    # x = random.uniform(0, 1)#首先随机生成一个0，1之间的随机数
    x = random.randint(1, sum(probabilities))
    cumulative_probability = 0
    for item, item_probability in zip(seq,probabilities):#seq代表待输入的字符串，prob代表各自字符串对应的概率
        cumulative_probability += item_probability#只有当累加的概率比刚才随机生成的随机数大时候，才跳出，并输出此时对应的字符串
        if x <= cumulative_probability:
            break
    return item  

In [3]:
def read_proteome_uniprot():
    '''
    Read the sequences of the proteins in the human proteome. 
    Sequence data are stored in a fasta file     
    Args:
        1. path: The input file containing sequence data of the proteome
            downloaded from the ensemble biomart FTP:
            ftp://ftp.ensembl.org/pub/release-90/fasta/homo_sapiens/.
    Return values:
        1. proteome: A dictionary whose keys are protein ensembl IDs
                    and values are protein sequences
    '''   
    path = "/home/jjia1/viralepitope/uniprot_proteins.csv"
    reference_df = pd.read_csv(path, index_col=0)
    reference_df1 = reference_df.set_index(['accession'])['seq'].to_dict()

    return reference_df1

In [4]:
uniprot = read_proteome_uniprot()

In [37]:
    iedb_csv = "/home/jjia1/viralepitope/viralepitope/iedb_data.txt"
    iedb_df = pd.read_csv(iedb_csv, sep='\t', skiprows=0, low_memory=False, dtype=object)
    iedb_df = np.array(iedb_df)
    
    all_positive_peptide = list(set([p[1] for p in iedb_df]))

In [34]:
    data_dict = {}
    for i in range(len(iedb_df)):
        allele = iedb_df[i][0]        
        if allele not in data_dict.keys():
            data_dict[allele] = [iedb_df[i].tolist()]
        else:
            data_dict[allele].append(iedb_df[i].tolist())

In [63]:
len(iedb_df)

176424

In [69]:
all_length_times

Counter({9: 153, 13: 3, 10: 8, 12: 3})

In [82]:
all_neg = []
for allele in data_dict.keys():
    # allele = 'HLA-B*41:06'
    traing_data = data_dict[allele]
    all_length = [len(traing_data[j][1]) for j in range(len(traing_data))]      
    all_length_times = Counter(all_length)

    all_probabilities = []
    for kmer in [8,9,10,11,12,13,14,15]:
        try:              
            probabilities = all_length_times[kmer]
        except:
            probabilities = 0   
        
        all_probabilities.append(probabilities)

In [79]:
all_length

[10]

In [83]:
def Random_peptides():
    '''
    Randomly sample peptides from the proteome
    Args:
        1. proteome: A dictionary of the human proteome.
        Output of the function read_proteome
    Return values:
        1. peptides: Sampled peptides.
    '''
    proteome = read_proteome_uniprot()
    
    iedb_csv = "/home/jjia1/viralepitope/viralepitope/iedb_data.txt"
    iedb_df = pd.read_csv(iedb_csv, sep='\t', skiprows=0, low_memory=False, dtype=object)
    iedb_df = np.array(iedb_df)
    
    all_positive_peptide = list(set([p[1] for p in iedb_df]))
    
    data_dict = {}
    for i in range(len(iedb_df)):
        allele = iedb_df[i][0]        
        if allele not in data_dict.keys():
            data_dict[allele] = [iedb_df[i].tolist()]
        else:
            data_dict[allele].append(iedb_df[i].tolist())

    #randomly generate peptides from the proteome    
    all_neg = []
    for allele in data_dict.keys():
        # allele = 'HLA-B*41:06'
        traing_data = data_dict[allele]
        all_length = [len(traing_data[j][1]) for j in range(len(traing_data))]      
        all_length_times = Counter(all_length)

        all_probabilities = []
        for kmer in [8,9,10,11,12,13,14,15]:
            try:              
                probabilities = all_length_times[kmer]
            except:
                probabilities = 0   
            
            all_probabilities.append(probabilities)
    
        pep_seq = []
        while len(pep_seq) < 10*len(traing_data):  #Set the number of random selections, here is 10 times the number of negative samples as positive samples
            length = random_pick_1([8,9,10,11,12,13,14,15],all_probabilities)  
            accession = random.choice(list(proteome.keys()))
            protein = proteome[accession]
            if len(protein) < length:
                    continue
            pep_start = random.randint(0, len(protein) - length)
            pep = protein[pep_start:pep_start + length]
            
            if set(list(pep)).difference(list('ACDEFGHIKLMNPQRSTVWY')):
                print('No offical peptide')
                continue       
                
            if pep in all_positive_peptide:
                print('In positive peptide')
                continue
                
            if pep not in pep_seq:
                pep_seq.append([accession, pep])
    
        for k in pep_seq:
            all_neg.append([allele, k[0], k[1]])
            
    return all_neg

In [84]:
neg = Random_peptides()

No offical peptide
No offical peptide
No offical peptide
No offical peptide
No offical peptide
No offical peptide
No offical peptide
No offical peptide
No offical peptide
No offical peptide
No offical peptide
No offical peptide
No offical peptide
No offical peptide
No offical peptide
No offical peptide
In positive peptide
No offical peptide
No offical peptide
No offical peptide
No offical peptide
No offical peptide
No offical peptide
No offical peptide
No offical peptide
In positive peptide
No offical peptide
No offical peptide
No offical peptide
No offical peptide
No offical peptide
No offical peptide
No offical peptide
No offical peptide
No offical peptide
In positive peptide
No offical peptide
No offical peptide
No offical peptide
No offical peptide
No offical peptide
No offical peptide
No offical peptide
No offical peptide
No offical peptide
In positive peptide
In positive peptide
No offical peptide
In positive peptide
In positive peptide
No offical peptide
In positive peptide
No o